In [1]:
!pip install transformers

In [2]:
__author__ = "@PBenavides"

import numpy as np
import pandas as pd
import torch
import transformers as ppb #Pytorch transformers
import re


if torch.cuda.is_available():
  dev = "cuda:0"
else:
  dev = "cpu"

device = torch.device(dev)

In [3]:
train = pd.read_csv('https://raw.githubusercontent.com/HackSpacePeru/Datasets_intro_Data_Science/master/Nlp_twitter/train_twitter_analysis.csv').iloc[:,1:]
test = pd.read_csv('https://raw.githubusercontent.com/HackSpacePeru/Datasets_intro_Data_Science/master/Nlp_twitter/test_twitter_analysis.csv').iloc[:,1:]
df = train.append(test,ignore_index=True)
print(train.shape, test.shape, df.shape)

(31962, 2) (17197, 1) (49159, 2)


In [4]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

In [5]:
train['tweet'] = train['tweet'].str.replace("@user","")
test['tweet'] = test['tweet'].str.replace("@user","")

In [6]:
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights).to(device) #Estamos moviendo este modelo al GPU para poder procesar los datos.

In [7]:
#Ahora procederemos a tokenizar el dataset. En este caso procesaremos todas las oraciones en una, como un batch. 
tokenized_train = train['tweet'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
tokenized_test = test['tweet'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

### Padding


In [8]:
#Para poder ver cuál es el tamaño máximo de palabras, correremos lo siguiente
max_len = 0
for i in tokenized_train.values:
  if len(i) > max_len:
    max_len = len(i)
print(max_len)

137


In [9]:
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized_train.values])
print(np.array(padded).shape)

(31962, 137)


### Masking
Si metemos directamente el objeto padded a Bert, lo confundirá un poco. Necesitamos crear otra variable para decirle que ignore el padding que agregamos cuando esté procesando el input. Es decir, que si hay un tweet con menos tamaño del max_len, que ponga un 0 en esa posición. Esto es como usar un máscara de atención, así que le pondremos attention_mask.

In [10]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(31962, 137)

Ahora procederemos a transformas las oraciones a los embeddings que tanto queremos con el Algoritmo DistilBERT.

In [11]:
#Cuando los convierto a tensores, estaré llevándolos luego al GPU para poder hacer el procesamiento.
input_ids = torch.tensor(padded).to(device)
attention_mask = torch.tensor(attention_mask).to(device)
print(input_ids.shape, attention_mask.shape)

torch.Size([31962, 137]) torch.Size([31962, 137])


Dado que Colab solo nos da 14Gbs de GPU, tendremos que acortar los 31mil tweets de train a solo 10mil, esto hará que tengamos una representación vectorial de esos 10mil tweets.

In [12]:
batch_step = 100
train_size = 12500 #Para elegir un número limitado de tweets, así haremos la prueba pues la memoria del GPU revienta.

import gc
embedding_tensor = torch.empty(size=(0,137,768)).to(device)

#El GPU de Colab no soporta tantos datos, recuerda que estamos hablando de 32000*137*768
with torch.no_grad():
  for batch_ in [(i,i+batch_step) for i in range(0,train_size,batch_step)]:

    last_hidden_states = model(input_ids[batch_[0]:batch_[1]])
    embedding_tensor = torch.cat([embedding_tensor, last_hidden_states[0]],dim=0)

    del last_hidden_states
    gc.collect()
    torch.cuda.empty_cache()

print("El embedding representa {} tweets con un máximo de {} palabras por tweet en {} números".format(embedding_tensor.shape[0], embedding_tensor.shape[1], embedding_tensor.shape[2]))

El embedding representa 12500 tweets con un máximo de 137 palabras por tweet en 768 números


#### Sobre el embedding:

Los embeddings en BERT tienen la característica de darnos un clasificador como primer número representativo. Esto se puede usar para clasificar luego nuestros tweets. Así que lo guardaremos en una variable **features**. Recuerda que este CLS nos sirve como representación total de cada doc que estamos poniendo en BERT. Es como una forma de reducir las dimensiones para poder tenerlo en una especie de tabla y hacer luego las predicciones con una regresión Logística.

In [16]:
features = embedding_tensor[:,0,:].cpu().numpy()
labels = train['label'].iloc[0:train_size]
print(features.shape, labels.shape)

(12500, 768) (12500,)


### Modelo de clasificación.

A partir de acá, ya tenemos los features. Solo nos concetraremos en mejorar el modelo para poder tener la mejor clasificación. En este caso, haremos un modelo logístico.

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(features, labels)
pipeline = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000))

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('logisticregression',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=1000,
                                    multi_class='auto', n_jobs=None,
                                    penalty='l2', random_state=None,
                                    solver='lbfgs', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=False)

In [22]:
print(pipeline.score(X_test, y_test))

0.95104
